In [1]:
import tensorflow as tf

In [2]:
!pip install -U tensorflow-addons

     |████████████████████████████████| 679 kB 7.5 MB/s 


In [3]:
import tensorflow_addons as tfa

In [53]:
mnist= tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test)= mnist.load_data()

x_train, x_test= x_train/255.0, x_test/255.0

In [64]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [40]:
import numpy as np
import os

In [65]:
batch1x= x_train[:30000]
batch2x= x_train[30000:]

batch1y= y_train[:30000]
batch2y=y_train[30000:]

test1x= x_test[:5000]
test1y= y_test[:5000]
test2x= x_test[5000:]
test2y= y_test[5000:]

In [66]:
print(batch1x.shape)
print(batch2x.shape)
print(test1x.shape)
print(test2x.shape)

(30000, 28, 28)
(30000, 28, 28)
(5000, 28, 28)
(5000, 28, 28)


In [15]:
batch2y

array([5, 9, 5, ..., 5, 6, 8], dtype=uint8)

In [67]:
batch1x = batch1x.reshape(batch1x.shape[0], 28, 28, 1)
test1x = test1x.reshape(test1x.shape[0], 28, 28, 1)
batch2x = batch2x.reshape(batch2x.shape[0], 28, 28, 1)
test2x = test2x.reshape(test2x.shape[0], 28, 28, 1)

In [68]:
batch1x = tf.image.resize(batch1x, [14,14])
test1x= tf.image.resize(test1x, [56,56])
test2x= tf.image.resize(test2x, [112, 122])

In [69]:
print(batch1x.shape)
print(batch2x.shape)
print(test1x.shape)
print(test2x.shape)

(30000, 14, 14, 1)
(30000, 28, 28, 1)
(5000, 56, 56, 1)
(5000, 112, 122, 1)


In [70]:
def create_model():
  model = tf.keras.models.Sequential([
  tfa.layers.SpatialPyramidPooling2D([1,4,12]),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
  ])

  model.compile(
    optimizer=tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
  ) 

  return model

In [71]:
model=create_model()
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(
    batch1x, batch1y, epochs=10, validation_data=(test1x,test1y),callbacks=[cp_callback])
model.summary()

!mkdir -p saved_model
model.save('saved_model/my_model')


#model.evaluate(x_test, y_test, verbose=2)

Epoch 1/10
938/938 [==============================] - 3s 2ms/step - loss: 0.5610 - sparse_categorical_accuracy: 0.8478 - val_loss: 0.3643 - val_sparse_categorical_accuracy: 0.9006

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.2800 - sparse_categorical_accuracy: 0.9188 - val_loss: 0.2803 - val_sparse_categorical_accuracy: 0.9244

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.2186 - sparse_categorical_accuracy: 0.9373 - val_loss: 0.2256 - val_sparse_categorical_accuracy: 0.9370

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.1814 - sparse_categorical_accuracy: 0.9472 - val_loss: 0.1970 - val_sparse_categorical_accuracy: 0.9454

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.1578 - s

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [72]:
model=create_model()
model.load_weights(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(
    batch2x, batch2y, epochs=10, validation_data=(test2x, test2y), callbacks=[cp_callback])
model.summary()

!mkdir -p saved_model
model.save('saved_model/my_model2')



Epoch 1/10
938/938 [==============================] - 4s 3ms/step - loss: 0.1305 - sparse_categorical_accuracy: 0.9609 - val_loss: 1.4892 - val_sparse_categorical_accuracy: 0.5620

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
938/938 [==============================] - 3s 3ms/step - loss: 0.1125 - sparse_categorical_accuracy: 0.9663 - val_loss: 1.5882 - val_sparse_categorical_accuracy: 0.5616

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0986 - sparse_categorical_accuracy: 0.9705 - val_loss: 1.4782 - val_sparse_categorical_accuracy: 0.5844

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0936 - sparse_categorical_accuracy: 0.9719 - val_loss: 1.5904 - val_sparse_categorical_accuracy: 0.5570

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0854 - s

INFO:tensorflow:Assets written to: saved_model/my_model2/assets


INFO:tensorflow:Assets written to: saved_model/my_model2/assets
